In [7]:
from keras import backend as K
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping

import pandas as pd
import numpy as np
import numpy
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

# seed 값 설정
seed = 0
numpy.random.seed(seed)
tf.set_random_seed(seed)


# 데이터 읽어 들이기--- (※1)
#ci_year	ci_term	si_grad	si_ent_gubun	agescale	si_sex	si_last_hak	schoarYn	
#accept_point	avg_logincnt	avg_att_time	avg_att_rate	avg_bbscnt	avg_startlogin	entYn	classcnt	
#avg_te_avg	avg_te_passrate	gradcnt	counsel	stdResult
#features =14
#features =13
#filename="./dataset/20200612_2018_2019.csv"
#filename="./dataset/20200614_v1.csv"
#filename="./dataset/20200717_v1.csv"

features =20
filename="./dataset/20200605_v1.csv"

df_pre=pd.read_csv(filename,encoding="cp949", header=0) # CSV파일을 불러오는 함수를 이용
#df_pre=pd.read_csv('./dataset/20200612_app_point.csv',encoding="cp949", header=0) # CSV파일을 불러오는 함수를 이용





In [8]:
print("데이터의 총 수", len(df_pre))



데이터의 총 수 58088


In [9]:
df_pre.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58088 entries, 0 to 58087
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ci_year              58088 non-null  int64  
 1   ci_term              58088 non-null  int64  
 2   si_grad              58088 non-null  int64  
 3   si_ent_gubun         58088 non-null  int64  
 4   agescale             58088 non-null  int64  
 5   si_sex               58088 non-null  int64  
 6   si_last_hak          58088 non-null  int64  
 7   schoarYn             58088 non-null  int64  
 8   accept_point         58088 non-null  int64  
 9   avg_logincnt         58088 non-null  int64  
 10  avg_att_time         58088 non-null  int64  
 11  avg_att_rate         58088 non-null  float64
 12  avg_bbscnt           58088 non-null  int64  
 13  avg_startlogin       58088 non-null  int64  
 14  entYn                58088 non-null  int64  
 15  classcnt             58088 non-null 

In [10]:

df_pre.tail()

,ci_year,ci_term,si_grad,si_ent_gubun,agescale,si_sex,si_last_hak,schoarYn,accept_point,avg_logincnt,...,avg_att_rate,avg_bbscnt,avg_startlogin,entYn,classcnt,pre_avg_te_avg,pre_avg_te_passrate,pre_gradcnt,counsel,stdResult
58083,2019,2,4,2,50,1,3,1,70,74,...,100.00,0,30,0,6,3.42,100,1,0,0
58084,2019,2,4,2,50,2,3,1,70,50,...,99.72,0,20,0,8,3.67,100,1,0,0
58085,2019,2,4,2,40,2,3,1,70,63,...,100.00,6,22,0,6,3.33,100,1,0,0
58086,2019,2,4,2,40,2,2,1,70,98,...,100.00,12,47,0,8,3.67,100,1,0,0
58087,2019,2,4,2,30,1,2,1,70,80,...,100.00,0,33,0,7,3.58,100,1,0,0


In [11]:
# 데이터 내부의 기호를 숫자로 변환하기--- (※2)
df = df_pre.sample(frac=1)


dataset = df.values
X = dataset[:,0:features]
Y = dataset[:,features]

# 학습 전용과 테스트 전용 데이터로 나누기 --- (※3)
#테스트, 검증데이터 분할 7:3 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=seed)

#데이터 정규화
X_train, X_test = X_train / 255, X_test / 255

from keras.layers import Dense, Dropout, Activation
# 모델 설정
model = Sequential()
model.add(Dense(28,  input_dim=features, activation='relu'))
model.add(Dense(15, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

#모델 컴파일
#model.compile(loss='binary_crossentropy',
model.compile(loss='mean_squared_error',
           optimizer='adam',
           # metrics=['accuracy'])
           metrics=['accuracy',recall_m,precision_m,f1_m])

# 학습 자동 중단 설정 성능이 개선되지 않는 epch 100번까지 수용
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=100)



# 모델 저장 폴더 설정
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
   os.mkdir(MODEL_DIR)

# 모델 저장 조건 설정
modelpath="./model/{epoch:02d}-{val_loss:.4f}.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)



# 모델 실행 및 저장
history = model.fit(X_train, Y_train, validation_split=0.20, epochs=500, batch_size=100, 
                    callbacks=[early_stopping_callback, checkpointer])

# 결과 출력
print("\n epoches-500, bat_size=100")
print("\n 학습중단 + 모델 성능개선 : early_stopping_callback:")
print("\n 예측정확도: %.4f" % (model.evaluate(X_test, Y_test)[1]))

Train on 32528 samples, validate on 8133 samples
Epoch 1/500
32528/32528 [==============================] - 1s 22us/step - loss: 0.2166 - accuracy: 0.6685 - recall_m: 0.9165 - precision_m: 0.6500 - f1_m: 0.7580 - val_loss: 0.2083 - val_accuracy: 0.7001 - val_recall_m: 1.0000 - val_precision_m: 0.7003 - val_f1_m: 0.8230

Epoch 00001: val_loss improved from inf to 0.20833, saving model to ./model/01-0.2083.hdf5
Epoch 2/500
32528/32528 [==============================] - 0s 15us/step - loss: 0.2035 - accuracy: 0.7038 - recall_m: 0.9948 - precision_m: 0.7049 - f1_m: 0.8242 - val_loss: 0.2033 - val_accuracy: 0.7042 - val_recall_m: 0.9690 - val_precision_m: 0.7130 - val_f1_m: 0.8205

Epoch 00002: val_loss improved from 0.20833 to 0.20329, saving model to ./model/02-0.2033.hdf5
Epoch 3/500
32528/32528 [==============================] - 0s 15us/step - loss: 0.2000 - accuracy: 0.7065 - recall_m: 0.9840 - precision_m: 0.7102 - f1_m: 0.8239 - val_loss: 0.2026 - val_accuracy: 0.7036 - val_recall_m:


Epoch 00023: val_loss did not improve from 0.14287
Epoch 24/500
32528/32528 [==============================] - 0s 14us/step - loss: 0.1438 - accuracy: 0.7981 - recall_m: 0.8700 - precision_m: 0.8498 - f1_m: 0.8573 - val_loss: 0.1443 - val_accuracy: 0.8008 - val_recall_m: 0.8163 - val_precision_m: 0.8906 - val_f1_m: 0.8510

Epoch 00024: val_loss did not improve from 0.14287
Epoch 25/500
32528/32528 [==============================] - 0s 14us/step - loss: 0.1443 - accuracy: 0.7972 - recall_m: 0.8683 - precision_m: 0.8515 - f1_m: 0.8566 - val_loss: 0.1515 - val_accuracy: 0.7701 - val_recall_m: 0.9320 - val_precision_m: 0.7819 - val_f1_m: 0.8496

Epoch 00025: val_loss did not improve from 0.14287
Epoch 26/500
32528/32528 [==============================] - 0s 15us/step - loss: 0.1429 - accuracy: 0.7996 - recall_m: 0.8685 - precision_m: 0.8525 - f1_m: 0.8577 - val_loss: 0.1395 - val_accuracy: 0.8086 - val_recall_m: 0.8521 - val_precision_m: 0.8718 - val_f1_m: 0.8610

Epoch 00026: val_loss im


Epoch 00047: val_loss improved from 0.12988 to 0.12959, saving model to ./model/47-0.1296.hdf5
Epoch 48/500
32528/32528 [==============================] - 0s 15us/step - loss: 0.1336 - accuracy: 0.8140 - recall_m: 0.8766 - precision_m: 0.8643 - f1_m: 0.8681 - val_loss: 0.1313 - val_accuracy: 0.8172 - val_recall_m: 0.8504 - val_precision_m: 0.8837 - val_f1_m: 0.8659

Epoch 00048: val_loss did not improve from 0.12959
Epoch 49/500
32528/32528 [==============================] - 0s 14us/step - loss: 0.1332 - accuracy: 0.8145 - recall_m: 0.8782 - precision_m: 0.8628 - f1_m: 0.8685 - val_loss: 0.1338 - val_accuracy: 0.8137 - val_recall_m: 0.8452 - val_precision_m: 0.8839 - val_f1_m: 0.8633

Epoch 00049: val_loss did not improve from 0.12959
Epoch 50/500
32528/32528 [==============================] - 0s 14us/step - loss: 0.1331 - accuracy: 0.8146 - recall_m: 0.8775 - precision_m: 0.8631 - f1_m: 0.8683 - val_loss: 0.1327 - val_accuracy: 0.8154 - val_recall_m: 0.8389 - val_precision_m: 0.8910 


Epoch 00096: val_loss improved from 0.12259 to 0.12197, saving model to ./model/96-0.1220.hdf5
Epoch 97/500
32528/32528 [==============================] - 0s 14us/step - loss: 0.1235 - accuracy: 0.8279 - recall_m: 0.9062 - precision_m: 0.8580 - f1_m: 0.8804 - val_loss: 0.1240 - val_accuracy: 0.8218 - val_recall_m: 0.8778 - val_precision_m: 0.8689 - val_f1_m: 0.8725

Epoch 00097: val_loss did not improve from 0.12197
Epoch 98/500
32528/32528 [==============================] - 0s 14us/step - loss: 0.1253 - accuracy: 0.8263 - recall_m: 0.9027 - precision_m: 0.8595 - f1_m: 0.8789 - val_loss: 0.1227 - val_accuracy: 0.8291 - val_recall_m: 0.8862 - val_precision_m: 0.8715 - val_f1_m: 0.8780

Epoch 00098: val_loss did not improve from 0.12197
Epoch 99/500
32528/32528 [==============================] - 0s 14us/step - loss: 0.1245 - accuracy: 0.8256 - recall_m: 0.9018 - precision_m: 0.8584 - f1_m: 0.8783 - val_loss: 0.1265 - val_accuracy: 0.8186 - val_recall_m: 0.8493 - val_precision_m: 0.8864 

32528/32528 [==============================] - 0s 15us/step - loss: 0.1207 - accuracy: 0.8361 - recall_m: 0.9092 - precision_m: 0.8661 - f1_m: 0.8857 - val_loss: 0.1226 - val_accuracy: 0.8320 - val_recall_m: 0.8734 - val_precision_m: 0.8851 - val_f1_m: 0.8785

Epoch 00121: val_loss did not improve from 0.11765
Epoch 122/500
32528/32528 [==============================] - 0s 14us/step - loss: 0.1203 - accuracy: 0.8367 - recall_m: 0.9084 - precision_m: 0.8670 - f1_m: 0.8860 - val_loss: 0.1231 - val_accuracy: 0.8287 - val_recall_m: 0.9292 - val_precision_m: 0.8421 - val_f1_m: 0.8829

Epoch 00122: val_loss did not improve from 0.11765
Epoch 123/500
32528/32528 [==============================] - 0s 14us/step - loss: 0.1182 - accuracy: 0.8422 - recall_m: 0.9127 - precision_m: 0.8701 - f1_m: 0.8900 - val_loss: 0.1310 - val_accuracy: 0.8167 - val_recall_m: 0.8317 - val_precision_m: 0.8988 - val_f1_m: 0.8630

Epoch 00123: val_loss did not improve from 0.11765
Epoch 124/500
32528/32528 [=========

...

32528/32528 [==============================] - 0s 11us/step - loss: 0.1059 - accuracy: 0.8565 - recall_m: 0.9241 - precision_m: 0.8786 - f1_m: 0.9001 - val_loss: 0.1042 - val_accuracy: 0.8580 - val_recall_m: 0.9218 - val_precision_m: 0.8808 - val_f1_m: 0.9002

Epoch 00394: val_loss did not improve from 0.10360
Epoch 395/500
32528/32528 [==============================] - 0s 12us/step - loss: 0.1079 - accuracy: 0.8550 - recall_m: 0.9211 - precision_m: 0.8789 - f1_m: 0.8987 - val_loss: 0.1064 - val_accuracy: 0.8553 - val_recall_m: 0.9170 - val_precision_m: 0.8806 - val_f1_m: 0.8979

Epoch 00395: val_loss did not improve from 0.10360
Epoch 396/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.1057 - accuracy: 0.8583 - recall_m: 0.9238 - precision_m: 0.8813 - f1_m: 0.9013 - val_loss: 0.1153 - val_accuracy: 0.8496 - val_recall_m: 0.9413 - val_precision_m: 0.8576 - val_f1_m: 0.8969

Epoch 00396: val_loss did not improve from 0.10360
Epoch 397/500
32528/32528 [=========


Epoch 00418: val_loss did not improve from 0.10314
Epoch 419/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.1055 - accuracy: 0.8562 - recall_m: 0.9234 - precision_m: 0.8790 - f1_m: 0.8998 - val_loss: 0.1303 - val_accuracy: 0.8229 - val_recall_m: 0.8527 - val_precision_m: 0.8893 - val_f1_m: 0.8699

Epoch 00419: val_loss did not improve from 0.10314
Epoch 420/500
32528/32528 [==============================] - 0s 12us/step - loss: 0.1065 - accuracy: 0.8560 - recall_m: 0.9229 - precision_m: 0.8790 - f1_m: 0.8996 - val_loss: 0.1082 - val_accuracy: 0.8575 - val_recall_m: 0.9544 - val_precision_m: 0.8577 - val_f1_m: 0.9029

Epoch 00420: val_loss did not improve from 0.10314
Epoch 421/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.1052 - accuracy: 0.8568 - recall_m: 0.9248 - precision_m: 0.8788 - f1_m: 0.9004 - val_loss: 0.1050 - val_accuracy: 0.8582 - val_recall_m: 0.9414 - val_precision_m: 0.8671 - val_f1_m: 0.9022

Epoch 00421: val_loss


Epoch 00443: val_loss improved from 0.10233 to 0.10218, saving model to ./model/443-0.1022.hdf5
Epoch 444/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.1057 - accuracy: 0.8561 - recall_m: 0.9234 - precision_m: 0.8787 - f1_m: 0.8997 - val_loss: 0.1046 - val_accuracy: 0.8566 - val_recall_m: 0.9336 - val_precision_m: 0.8705 - val_f1_m: 0.9004

Epoch 00444: val_loss did not improve from 0.10218
Epoch 445/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.1061 - accuracy: 0.8566 - recall_m: 0.9254 - precision_m: 0.8773 - f1_m: 0.8997 - val_loss: 0.1063 - val_accuracy: 0.8569 - val_recall_m: 0.9354 - val_precision_m: 0.8697 - val_f1_m: 0.9007

Epoch 00445: val_loss did not improve from 0.10218
Epoch 446/500
32528/32528 [==============================] - 0s 13us/step - loss: 0.1046 - accuracy: 0.8577 - recall_m: 0.9243 - precision_m: 0.8799 - f1_m: 0.9008 - val_loss: 0.1157 - val_accuracy: 0.8499 - val_recall_m: 0.8903 - val_precision_m: 0.8


Epoch 00468: val_loss did not improve from 0.10159
Epoch 469/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.1049 - accuracy: 0.8573 - recall_m: 0.9273 - precision_m: 0.8773 - f1_m: 0.9008 - val_loss: 0.1029 - val_accuracy: 0.8580 - val_recall_m: 0.9263 - val_precision_m: 0.8774 - val_f1_m: 0.9005

Epoch 00469: val_loss did not improve from 0.10159
Epoch 470/500
32528/32528 [==============================] - 0s 12us/step - loss: 0.1044 - accuracy: 0.8571 - recall_m: 0.9258 - precision_m: 0.8775 - f1_m: 0.9002 - val_loss: 0.1039 - val_accuracy: 0.8564 - val_recall_m: 0.9271 - val_precision_m: 0.8748 - val_f1_m: 0.8996

Epoch 00470: val_loss did not improve from 0.10159
Epoch 471/500
32528/32528 [==============================] - 0s 12us/step - loss: 0.1065 - accuracy: 0.8556 - recall_m: 0.9255 - precision_m: 0.8766 - f1_m: 0.8993 - val_loss: 0.1035 - val_accuracy: 0.8591 - val_recall_m: 0.9325 - val_precision_m: 0.8742 - val_f1_m: 0.9018

Epoch 00471: val_loss

32528/32528 [==============================] - 0s 11us/step - loss: 0.1037 - accuracy: 0.8591 - recall_m: 0.9290 - precision_m: 0.8784 - f1_m: 0.9022 - val_loss: 0.1038 - val_accuracy: 0.8576 - val_recall_m: 0.9280 - val_precision_m: 0.8756 - val_f1_m: 0.9004

Epoch 00494: val_loss did not improve from 0.10159
Epoch 495/500
32528/32528 [==============================] - 0s 13us/step - loss: 0.1050 - accuracy: 0.8579 - recall_m: 0.9283 - precision_m: 0.8778 - f1_m: 0.9015 - val_loss: 0.1076 - val_accuracy: 0.8554 - val_recall_m: 0.9122 - val_precision_m: 0.8847 - val_f1_m: 0.8976

Epoch 00495: val_loss did not improve from 0.10159
Epoch 496/500
32528/32528 [==============================] - 0s 12us/step - loss: 0.1044 - accuracy: 0.8577 - recall_m: 0.9279 - precision_m: 0.8773 - f1_m: 0.9011 - val_loss: 0.1049 - val_accuracy: 0.8580 - val_recall_m: 0.9350 - val_precision_m: 0.8713 - val_f1_m: 0.9014

Epoch 00496: val_loss did not improve from 0.10159
Epoch 497/500
32528/32528 [=========

In [12]:
#테스트 데이터 검증
loss, accuracy, recall,precision,f1_socre  = model.evaluate(X_test, Y_test)
#accuracy  = model.evaluate(X_test, Y_test)

print("\n accuracy, precision, recall, f1_score")
print("\n  %.4f, %.4f, %4f, %.4f" % (accuracy, precision ,recall, f1_socre))

#print("\n accuracy")
#print("\n  %.4f % (accuracy)")

17427/17427 [==============================] - 0s 14us/step

 accuracy, precision, recall, f1_score

  0.8513, 0.8850, 0.904231, 0.8924
